In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# relevant modules for code
import neurd
from mesh_tools import trimesh_utils as tu
from datasci_tools import ipyvolume_utils as ipvu
from neurd import neuron_visualizations as nviz
from pathlib import Path
import os

import inspect
print(inspect.getfile(tu))



from datasci_tools import pipeline



/usr/local/lib/python3.8/dist-packages/python_jsonschema_objects/__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


/usr/local/lib/python3.8/dist-packages/mesh_tools/trimesh_utils.py


Use MeshParty to Upload ChenLab Data 

In [3]:
#  ChenLab script...

from meshparty import trimesh_io  # for grabbing a mesh
from meshparty import trimesh_vtk  # for visualizing meshes and skeletons
    
# provide the DENDRITIC segmentation ID for the cell of interest
dend_id = 75719036679410195  

# Get the DENDRITIC segmentation mesh from neuroglancer
mm = trimesh_io.MeshMeta(
        cv_path="precomputed://gs://zetta_jchen_mouse_cortex_001_segmentation/cell/lowres/240820_emb24_lowres_x1/seg", 
        map_gs_to_https=True)

mesh = mm.mesh(seg_id=dend_id) 

# visualize the mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh,  color=(1,0,0),  opacity=0.25)
#trimesh_vtk.render_actors([mesh_actor]) #NOTE: need to close the window when done to unpause python

# store the mesh
# Define the local file path where you want to save the mesh (with .off extension)
local_file_path = "32414.off"
# Export the mesh to a .off file
mesh.export(local_file_path)
# Confirm the file has been saved
print(f"Mesh has been saved to {os.path.abspath(local_file_path)}")


#import skeletor as sk
# pre-process the mesh (may not be necessary)
#mesh_fix = sk.pre.fix_mesh(mesh) #
#mesh_fix_actor = trimesh_vtk.mesh_actor(mesh_fix,  color=(1,0,0),  opacity=0.25)
#trimesh_vtk.render_actors([mesh_actor, mesh_fix_actor]) 



Mesh has been saved to /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/32414.off


Process ForkPoolWorker-4:
Process ForkPoolWorker-6:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-5:
Process ForkPoolWorker-2:
Process ForkPoolWorker-10:
Process ForkPoolWorker-7:
Process ForkPoolWorker-9:
Traceback (most recent call last):
Process ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.8/dist-packages/multiprocess/process.py", line 315, in _b

# Load the appropriate volume data inferface object

In [4]:
from neurd.vdi_microns import volume_data_interface as vdi
vdi.set_parameters_for_directory_modules()

# Data Prep: Loading mesh

In [5]:
# ChenLab MESH!!!
segment_id = 32414

mesh = vdi.fetch_segment_id_mesh(
    segment_id,
    plot = True
)

mesh

/usr/local/lib/python3.8/dist-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

<trimesh.Trimesh(vertices.shape=(365198, 3), faces.shape=(730572, 3), name=`32414.off`)>

In [6]:
# analyze dataset properties (ChenLab)

filepath = '32414.off'
print(f'mesh size {os.path.getsize(filepath)} bytes')


mesh size 36116418 bytes


In [7]:
from datasci_tools import pipeline

products = pipeline.PipelineProducts()
products

# Step 1: Decimation

Decimation involves


In [8]:
"""
NOTE: an already decimated mesh is used for the example
in order to speed up time, that's why the mesh_dec
object will be overwritten with original mesh object
"""

"\nNOTE: an already decimated mesh is used for the example\nin order to speed up time, that's why the mesh_dec\nobject will be overwritten with original mesh object\n"

In [9]:
decimation_parameters = dict(
    decimation_ratio =0.25,
)

In [10]:
import time

# tu

st = time.time()
mesh_decimated = tu.decimate(
    mesh,
    **decimation_parameters
)

print(f"Total time = {time.time() - st}")

xvfb-run -n 9959 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_6595.off -o /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_6595_decimated.off -s /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_25845122.mls
Total time = 22.991239070892334


In [11]:
# saving any parameters used or output products for later use
products.set_stage_attrs(
    stage = "decimation",
    attr_dict = dict(
        decimation_parameters = decimation_parameters,
        segment_id = segment_id,
    ),
)

print(products)

{
  'decimation':
  {
    'decimation_parameters':
    {
      'decimation_ratio':0.25,
    },
    'segment_id':32414,
  },
}



# Step 2: Soma Identification

In [12]:
from neurd import soma_extraction_utils as sm
soma_extraction_parameters = dict()

In [13]:
# could inspect the parameters of a certain category
from neurd import parameter_utils as paru
paru.category_param_from_module(
        module = sm,
        category = "glia",
)

{'glia_volume_threshold_in_um_global': 2500,
 'glia_n_faces_threshold_global': 400000,
 'glia_n_faces_min_global': 100000}

In [14]:
from neurd import neuron_visualizations as nviz

verbose = True

soma_products = sm.soma_indentification(
    mesh_decimated,
    verbose=verbose,
    **soma_extraction_parameters
)

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 5000.0 
large_mesh_threshold_inner = 3250.0 
soma_size_threshold = 562.5 
soma_size_threshold_max = 75000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
max_mesh_sized_filtered_away = 22500.0
xvfb-run -n 5925 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_47774.off -o /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_47774_remove_interior.off -s /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/remove_interior_829214.mls
removed temporary input file: /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_47774.off
removed temporary output file: /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_47774_remove_interior.off
/NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_S

  0%|          | 0/14 [00:00<?, ?it/s]

      ------ Found 1 viable somas: [0.8670964999999999]
Using Poisson Surface Reconstruction for watertightness in soma_volume_ratio
xvfb-run -n 6739 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/Poisson_temp/neuron_903877.off -o /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/Poisson_temp/neuron_903877_poisson.off -s /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/Poisson_temp/poisson_622956.mls
removed temporary input file: /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/Poisson_temp/neuron_903877.off
removed temporary output file: /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/Poisson_temp/neuron_903877_poisson.off
mesh.is_watertight = True
/NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/Poisson_temp/poisson_622956.mls is being deleted....
Inside sphere validater: ratio_val = 2.48

  0%|          | 0/5 [00:00<?, ?it/s]

      ------ Found 1 viable somas: [0.577805]
xz = 6.232575437305518 ratio was beyong 6 multiplier
yz = 7.309074290114619 ratio was beyong 6 multiplier
mesh.is_watertight = True
/NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/Poisson_temp/poisson_692513.mls is being deleted....
Inside sphere validater: ratio_val = 455.11247054898183
->Attempting retry of soma because failed first checks: soma_mesh = <trimesh.Trimesh(vertices.shape=(542, 3), faces.shape=(1077, 3), name=`neuron_12345_decimated_largest_piece_poisson_largest_inner_decimated.off`)>, curr_side_len_check = False, curr_volume_check = False
perc_0_faces = 0.002785515320334262
Going to run cgal segmentation with:
File: /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/110_mesh 
clusters:3 
smoothness:0.2

Number of segments: 3



  0%|          | 0/3 [00:00<?, ?it/s]

yz = 6.041267222886148 ratio was beyong 6 multiplier
mesh.is_watertight = True
/NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/Poisson_temp/poisson_103262.mls is being deleted....
Inside sphere validater: ratio_val = 393.3269542168001
--->This soma mesh was not added because failed retry of sphere validation:
 soma_mesh = <trimesh.Trimesh(vertices.shape=(457, 3), faces.shape=(908, 3), name=`neuron_12345_decimated_largest_piece_poisson_largest_inner_decimated.off`)>, curr_side_len_check = False, curr_volume_check = False



 Total time for run = 55.14832139015198
Before Filtering the number of somas found = 1

---Performing Soma Mesh Backtracking to original mesh for poisson soma 0
backtrack_soma_size_threshold = 8000
xvfb-run -n 5801 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_3250.off -o /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron

  0%|          | 0/1 [00:00<?, ?it/s]

Largest hole before segmentation = 131857.2229139092, after = 131857.2229139092,

ratio = 1.0, difference = 0.0
filtered_soma_list_components = [<trimesh.Trimesh(vertices.shape=(8836, 3), faces.shape=(17487, 3))>]
Not need to do a second pass because already found a soma


In [15]:
sm.plot_soma_products(
    mesh_decimated,
    soma_products = soma_products,
    verbose = True
)

# of somas = 1
# of glia = 0
# of nuclei = 1


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

In [16]:
soma_products.soma_extraction_parameters = soma_extraction_parameters
soma_products

In [17]:
products.set_stage_attrs(
    stage = "soma_identification",
    attr_dict = soma_products,
)

print(products)

{
  'decimation':
  {
    'decimation_parameters':
    {
      'decimation_ratio':0.25,
    },
    'segment_id':32414,
  },
  'soma_identification':
  {
    'glia_meshes':[],
    'nuclei_meshes':[<trimesh.Trimesh(vertices.shape=(4492, 3), faces.shape=(8984, 3))>],
    'soma_extraction_parameters':
    {
    },
    'soma_meshes':[<trimesh.Trimesh(vertices.shape=(8836, 3), faces.shape=(17487, 3))>],
    'soma_run_time':55.14832043647766,
    'soma_sdfs':array([0.8670965]),
  },
}



# Step 2b: Saving off pipeline products

In [18]:
from datasci_tools import system_utils as su
su.save_object(
    products,"products_up_to_soma_stage"
)

Saved object at /NEURD_Chenlab/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/products_up_to_soma_stage.pkl
File size is 7.846925 MB
Cleared mesh: <trimesh.Trimesh(vertices.shape=(4492, 3), faces.shape=(8984, 3))>:140171624630832
Cleared mesh: <trimesh.Trimesh(vertices.shape=(8836, 3), faces.shape=(17487, 3))>:140175250193184


# Step 3: Decomposition

In [19]:
import networkx as nx
nx.__version__

'2.8.8'

In [20]:
from neurd import neuron

neuron_obj = neuron.Neuron(
    mesh = mesh,
    segment_id = segment_id, # don't need this explicitely if segment_id is already in products
    pipeline_products = products,
    suppress_preprocessing_print=False,
    suppress_output=False,
)

neuron_obj

Processing Neuorn in minimal output mode...please wait


KeyboardInterrupt: 

In [ ]:
nviz.visualize_neuron(
    neuron_obj,
    limb_branch_dict="all"
)

# Calculate statistics

In [ ]:
decomp_products = neuron_obj.calculate_decomposition_products(
    store_in_obj = True,
)

decomp_products

In [ ]:
print(neuron_obj.pipeline_products)

# Saving off neuron

In [ ]:
vdi.save_neuron_obj(
    neuron_obj,
    verbose = True
)

# Reloading the Neuron

In [ ]:
neuron_obj_rec = vdi.load_neuron_obj(
    segment_id = segment_id,
    #mesh_decimated = mesh_decimated
)

In [ ]:
nviz.visualize_neuron(
    neuron_obj_rec,
    limb_branch_dict="all"
)

In [ ]:
print(neuron_obj_rec.pipeline_products)

# Exporting Branch meshes

In [ ]:
nviz.plot_limb(neuron_obj_rec,0)

In [ ]:
branch_idx = 3
limb_idx = 0

branch_mesh = neuron_obj_rec[limb_idx][branch_idx].mesh
ipvu.plot_objects(branch_mesh)

In [ ]:
_ = branch_mesh.export(f"{segment_id}_limb_{limb_idx}_branch_{branch_idx}.off")

# Step 4A: Multi Soma Split Suggestions

In [ ]:
from neurd import neuron_visualizations as nviz
nviz.plot_soma_limb_concept_network(
    neuron_obj_rec
)

In [ ]:
multi_soma_split_parameters = dict()

In [ ]:
_ = neuron_obj_rec.calculate_multi_soma_split_suggestions(
    plot = True,
    store_in_obj = True,
    **multi_soma_split_parameters
)


In [ ]:
neuron_obj_rec.pipeline_products.multi_soma_split_suggestions.multi_soma_split_parameters = multi_soma_split_parameters

# Step 4B: Multi Soma Split Execution

In [ ]:
neuron_list = neuron_obj_rec.multi_soma_split_execution(
    verbose = False,
)

In [ ]:
n1 = neuron_list[0]
print(n1.pipeline_products)

In [ ]:
type(n1.pipeline_products)

# Step 5: Cell Typing

In [ ]:
from neurd import neuron_pipeline_utils as npu

In [ ]:
neuron_obj_axon = npu.cell_type_ax_dendr_stage(
    n1,
    mesh_decimated = mesh_decimated,
    plot_axon = False,
)

In [ ]:
nviz.plot_axon(
    neuron_obj_axon
)

# Autoproofreading Stage

In [ ]:
neuron_obj_proof = npu.auto_proof_stage(
    neuron_obj_axon,
    mesh_decimated = mesh_decimated,
    calculate_after_proof_stats = False,
)

In [ ]:
_ = npu.after_auto_proof_stats(
    neuron_obj_proof,
    store_in_obj = True,
)

# Saving off the autoproofread neuron

In [ ]:
vdi.save_neuron_obj_auto_proof(
    neuron_obj_proof,
)

# Plotting the final neuron

In [ ]:
from neurd import synapse_utils as syu

In [ ]:
from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    neuron_obj_axon.mesh,
    meshes = [neuron_obj_proof.mesh],
    meshes_colors="red"
)

In [ ]:
syu.plot_synapses(neuron_obj_proof)

In [ ]:
syu.plot_synapses_error_from_neuron_obj(neuron_obj_proof)    

In [ ]:
syu.plot_synapses_valid_from_neuron_obj(neuron_obj_proof)  

In [ ]:
nviz.plot_compartments(neuron_obj_proof)

In [ ]:
%load_ext autoreload
%autoreload 2